In [103]:
# Parameters:
# Set test_run to True only when running tests. Keep it as False (default) for normal use
test_run= True #False

In [112]:
from dataclasses import dataclass, asdict
import time

@dataclass
class LogMessage():
    message: str
    job_name: str
    destination_database_name: str = ""
    destination_table_name: str = ""
    source_table_name: str =""
    duration: str = "0"
    job_start_time: str = ""
    job_end_time: str = ""

base_message = None

def print_message(start=False, stop=False):
    msg_str = '{"message": "' + base_message.message + '", "job_name": "' + base_message.job_name + '"'
    if start or stop:
        msg_str += ', "destination": "' + base_message.destination_database_name + '.' + base_message.destination_table_name + '"'
    if stop:
         msg_str += ', "duration": "' + base_message.duration + '"'
    msg_str += '}'
    return msg_str

spark_log4j = sc._jvm.org.apache.log4j
logger = spark_log4j.LogManager.getLogger("synapse_dataplatform")

In [105]:
def start_logging(job_name, destination_database_name, destination_table_name, source_table_name):
    job_start_time = time.perf_counter()
    
    global base_message
    base_message = LogMessage(message="", job_name=job_name, destination_database_name=destination_database_name, destination_table_name=destination_table_name, job_start_time=job_start_time)

    base_message.message=f"Starting logger for {job_name}"
    logger.info(print_message(start=True))

In [106]:
def stop_logging(job_name):
    base_message.job_end_time = time.perf_counter()
    base_message.duration = str(base_message.job_end_time - base_message.job_start_time)
    base_message.message = f"Stopping logger for {job_name}"
    logger.info(print_message(stop=True))

In [107]:
def log_informational_message(message):
    base_message.message = message
    logger.info(print_message())

In [108]:
def log_warning_message(message):
    base_message.message = message
    logger.warn(print_message())

In [109]:
def log_error_message(message):
    base_message.message = message
    logger.error(print_message())

In [113]:
if test_run:
    start_logging('log_test_job', 'log_test_destination_db', 'log_test_destination_table', 'log_test_source_table')
    log_informational_message('Log test informational message')
    stop_logging('log_test_job')